# Web Fundamentals Overview

Core browser concepts, rendering pipeline, JavaScript execution model, and essential APIs for frontend development.

## Browser Rendering Pipeline

The browser transforms HTML, CSS, and JavaScript into pixels on screen through a multi-stage pipeline.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        BROWSER RENDERING PIPELINE                           │
└─────────────────────────────────────────────────────────────────────────────┘

     HTML                           CSS
       │                             │
       ▼                             ▼
  ┌─────────┐                  ┌──────────┐
  │  Parse  │                  │  Parse   │
  │  HTML   │                  │   CSS    │
  └────┬────┘                  └────┬─────┘
       │                             │
       ▼                             ▼
  ┌─────────┐                  ┌──────────┐
  │   DOM   │                  │  CSSOM   │
  │  Tree   │                  │   Tree   │
  └────┬────┘                  └────┬─────┘
       │                             │
       └──────────┬──────────────────┘
                  ▼
           ┌─────────────┐
           │ Render Tree │  (visible elements + computed styles)
           └──────┬──────┘
                  │
       ┌──────────┼──────────┐
       ▼          ▼          ▼
  ┌────────┐ ┌────────┐ ┌────────┐
  │ Layout │ │ Paint  │ │Composite│
  │(Reflow)│ │        │ │ Layers │
  └────────┘ └────────┘ └────────┘
       │          │          │
       └──────────┼──────────┘
                  ▼
            ┌──────────┐
            │  Pixels  │
            │ on Screen│
            └──────────┘
```

### Key Components

| Component | Description | Trigger for Recalculation |
|-----------|-------------|---------------------------|
| **DOM** | Document Object Model - tree of HTML elements | HTML changes, `innerHTML`, `appendChild` |
| **CSSOM** | CSS Object Model - tree of styles | Stylesheet changes, inline styles |
| **Render Tree** | Combined DOM + CSSOM (only visible elements) | DOM/CSSOM changes |
| **Layout** | Calculate position and size of elements | Geometry changes (width, height, position) |
| **Paint** | Fill pixels (colors, borders, shadows) | Visual changes (color, visibility) |
| **Composite** | Layer management and GPU acceleration | `transform`, `opacity` changes |

## Critical Rendering Path (CRP)

The sequence of steps browser takes to convert HTML/CSS/JS into rendered pixels. Optimizing CRP improves time-to-first-paint.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    CRITICAL RENDERING PATH                              │
└─────────────────────────────────────────────────────────────────────────┘

  Request    Receive     Parse      Render      Layout     Paint
  HTML       HTML        HTML       Tree        Tree       Pixels
    │          │           │          │           │          │
    ▼          ▼           ▼          ▼           ▼          ▼
────●──────────●───────────●──────────●───────────●──────────●────▶ Time
    │          │           │          │           │          │
    │          │     ┌─────┴─────┐    │           │          │
    │          │     │           │    │           │          │
    │          │ CSS blocked  JS blocked         │          │
    │          │ (render      (parser            │          │
    │          │  blocking)    blocking)         │          │

BLOCKING RESOURCES:
────────────────────
  CSS ─────▶ Render Blocking  (delays first paint)
  JS  ─────▶ Parser Blocking  (halts DOM construction)
```

### Optimization Strategies

```html
<!-- Defer non-critical JS (loads async, executes after parse) -->
<script src="app.js" defer></script>

<!-- Async JS (loads async, executes immediately when ready) -->
<script src="analytics.js" async></script>

<!-- Preload critical resources -->
<link rel="preload" href="critical.css" as="style">
<link rel="preload" href="hero.jpg" as="image">

<!-- Inline critical CSS for above-the-fold content -->
<style>/* Critical CSS here */</style>

<!-- Lazy load non-critical CSS -->
<link rel="stylesheet" href="non-critical.css" media="print" onload="this.media='all'">
```

## JavaScript Event Loop

JavaScript is single-threaded but achieves concurrency through the event loop mechanism.

```
┌────────────────────────────────────────────────────────────────────────────────┐
│                          JAVASCRIPT EVENT LOOP                                 │
└────────────────────────────────────────────────────────────────────────────────┘

                    ┌─────────────────────────────────────────┐
                    │            CALL STACK                   │
                    │  ┌─────────────────────────────────┐   │
                    │  │   function c() { ... }          │   │
                    │  ├─────────────────────────────────┤   │
                    │  │   function b() { c() }          │   │
                    │  ├─────────────────────────────────┤   │
                    │  │   function a() { b() }          │   │
                    │  └─────────────────────────────────┘   │
                    └───────────────────┬─────────────────────┘
                                        │
     ┌──────────────────────────────────┼──────────────────────────────────┐
     │                                  │                                  │
     │                          ┌───────▼───────┐                          │
     │                          │  EVENT LOOP   │                          │
     │                          │   (checks)    │◄─────────────────┐       │
     │                          └───────┬───────┘                  │       │
     │                                  │                          │       │
     │              ┌───────────────────┴───────────────────┐      │       │
     │              ▼                                       ▼      │       │
     ▼                                                             │       │
┌─────────────────────────┐                    ┌─────────────────────────┐ │
│   MICROTASK QUEUE       │ ◄── Priority 1    │   MACROTASK QUEUE       │ │
│   (Higher Priority)     │                    │   (Lower Priority)      │ │
├─────────────────────────┤                    ├─────────────────────────┤ │
│ • Promise.then()        │                    │ • setTimeout()          │ │
│ • queueMicrotask()      │                    │ • setInterval()         │ │
│ • MutationObserver      │                    │ • I/O callbacks         │ │
│ • async/await           │                    │ • UI rendering          │ │
└────────────┬────────────┘                    │ • requestAnimationFrame │ │
             │                                 └────────────┬────────────┘ │
             │                                              │              │
             └──────────────────────────────────────────────┴──────────────┘

EXECUTION ORDER:
─────────────────
  1. Execute current script (synchronous code)
  2. Empty ALL microtasks (until queue is empty)
  3. Execute ONE macrotask
  4. Repeat from step 2
```

### Example Execution Order

```javascript
console.log('1');                          // Sync
setTimeout(() => console.log('2'), 0);     // Macrotask
Promise.resolve().then(() => console.log('3'));  // Microtask
console.log('4');                          // Sync

// Output: 1, 4, 3, 2
```

## CSS Box Model

Every element is rendered as a rectangular box with four layers.

```
┌─────────────────────────────────────────────────────────────────┐
│                           MARGIN                                │
│   ┌─────────────────────────────────────────────────────────┐   │
│   │                         BORDER                          │   │
│   │   ┌─────────────────────────────────────────────────┐   │   │
│   │   │                     PADDING                     │   │   │
│   │   │   ┌─────────────────────────────────────────┐   │   │   │
│   │   │   │                                         │   │   │   │
│   │   │   │              CONTENT                    │   │   │   │
│   │   │   │           (width × height)              │   │   │   │
│   │   │   │                                         │   │   │   │
│   │   │   └─────────────────────────────────────────┘   │   │   │
│   │   │                                                 │   │   │
│   │   └─────────────────────────────────────────────────┘   │   │
│   │                                                         │   │
│   └─────────────────────────────────────────────────────────┘   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

BOX-SIZING COMPARISON:
──────────────────────
  content-box (default):  width = content only
  border-box:             width = content + padding + border
```

```css
/* Modern reset - use border-box everywhere */
*, *::before, *::after {
  box-sizing: border-box;
}

.element {
  width: 300px;         /* Total width with border-box */
  padding: 20px;
  border: 5px solid;
  margin: 10px;
  /* Actual content width: 300 - 40 - 10 = 250px */
}
```

## Responsive Design

### Media Queries

```css
/* Mobile First Approach (recommended) */
.container { width: 100%; }           /* Mobile default */

@media (min-width: 768px) {           /* Tablet */
  .container { width: 750px; }
}

@media (min-width: 1024px) {          /* Desktop */
  .container { width: 970px; }
}

/* Feature queries */
@media (hover: hover) { }              /* Device has hover */
@media (prefers-color-scheme: dark) { } /* Dark mode */
@media (prefers-reduced-motion: reduce) { } /* Accessibility */
```

### Flexbox vs Grid

```
┌────────────────────────────────────────────────────────────────────────────┐
│  FLEXBOX (1D - Row OR Column)         │  GRID (2D - Rows AND Columns)      │
├───────────────────────────────────────┼────────────────────────────────────┤
│                                       │                                    │
│  ┌──────┬──────┬──────┬──────┐       │  ┌──────┬──────┬──────┐            │
│  │  1   │  2   │  3   │  4   │       │  │  1   │  2   │  3   │            │
│  └──────┴──────┴──────┴──────┘       │  ├──────┼──────┼──────┤            │
│        ─────▶ main axis              │  │  4   │  5   │  6   │            │
│                                       │  ├──────┼──────┼──────┤            │
│  Use for:                            │  │  7   │  8   │  9   │            │
│  • Navigation bars                   │  └──────┴──────┴──────┘            │
│  • Card content alignment            │                                    │
│  • Centering elements                │  Use for:                          │
│  • Dynamic sizing in one direction   │  • Page layouts                    │
│                                       │  • Card grids                      │
│                                       │  • Complex 2D arrangements         │
└───────────────────────────────────────┴────────────────────────────────────┘
```

```css
/* Flexbox */
.flex-container {
  display: flex;
  justify-content: space-between;  /* Main axis */
  align-items: center;             /* Cross axis */
  gap: 1rem;
}

/* Grid */
.grid-container {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
  gap: 1rem;
}
```

## Browser APIs

### Web Storage API

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         WEB STORAGE COMPARISON                              │
├─────────────────────┬─────────────────────────┬─────────────────────────────┤
│                     │     localStorage        │     sessionStorage          │
├─────────────────────┼─────────────────────────┼─────────────────────────────┤
│ Lifetime            │ Persistent (no expiry)  │ Tab/window session only     │
│ Capacity            │ ~5-10 MB                │ ~5-10 MB                    │
│ Scope               │ Origin (protocol+domain)│ Origin + tab                │
│ Sent with requests  │ No                      │ No                          │
│ Access              │ Synchronous             │ Synchronous                 │
└─────────────────────┴─────────────────────────┴─────────────────────────────┘
```

```javascript
// localStorage (persists across sessions)
localStorage.setItem('user', JSON.stringify({ name: 'John' }));
const user = JSON.parse(localStorage.getItem('user'));
localStorage.removeItem('user');
localStorage.clear();

// sessionStorage (cleared when tab closes)
sessionStorage.setItem('tempData', 'value');
```

### Fetch API

```javascript
// GET request
const response = await fetch('/api/users');
if (!response.ok) throw new Error(`HTTP ${response.status}`);
const data = await response.json();

// POST request with error handling
try {
  const response = await fetch('/api/users', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({ name: 'John' }),
    signal: AbortSignal.timeout(5000)  // 5s timeout
  });
  const result = await response.json();
} catch (error) {
  if (error.name === 'AbortError') console.log('Request timed out');
}
```

### History API

```javascript
// Navigate without page reload (SPA routing)
history.pushState({ page: 1 }, '', '/page1');   // Add to history
history.replaceState({ page: 2 }, '', '/page2'); // Replace current

// Handle back/forward navigation
window.addEventListener('popstate', (event) => {
  console.log('State:', event.state);  // { page: 1 }
  // Re-render based on state
});

// Navigation
history.back();     // Go back
history.forward();  // Go forward
history.go(-2);     // Go back 2 pages
```

## Quick Reference

| Topic | Key Concepts | Performance Impact |
|-------|-------------|--------------------| 
| **DOM** | Tree structure, `document.querySelector` | Batch DOM reads/writes |
| **CSSOM** | Cascading, specificity, inheritance | Avoid expensive selectors |
| **Layout** | Reflow triggers: width, height, position | Use `transform` instead |
| **Paint** | Repaint triggers: color, visibility | Promote to compositor layer |
| **Event Loop** | Microtasks before macrotasks | Don't block main thread |
| **CRP** | Defer JS, inline critical CSS | Minimize render-blocking |

### Performance Best Practices

```javascript
// ❌ Bad: Forces layout thrashing (read-write-read-write)
elements.forEach(el => {
  const height = el.offsetHeight;  // Read (forces layout)
  el.style.height = height + 10 + 'px';  // Write
});

// ✅ Good: Batch reads, then batch writes
const heights = elements.map(el => el.offsetHeight);  // All reads
elements.forEach((el, i) => {
  el.style.height = heights[i] + 10 + 'px';  // All writes
});

// ✅ Use CSS transforms for animations (GPU accelerated)
.animate { transform: translateX(100px); }  // Good
.animate { left: 100px; }                   // Triggers layout
```